<a href="https://colab.research.google.com/github/marcinwolter/Tracking_student2020/blob/master/GNN_MuonE_v1_Robust_mc_recon_comparison.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tutorial: GNNs for Particle Tracking

HEP.TrkX group

Steve Farrell, Daniel Murname

*Feb 2020*

Adapted for MuonE tracking by Marcin Wolter

*July 2020*

In [1]:
# System imports
import os
import sys
from pprint import pprint as pp
from time import time as tt

# External imports
import matplotlib.pyplot as plt
import matplotlib.colors
import numpy as np
import pandas as pd
import seaborn as sns
import torch

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
print("PyTorch version:",print(torch.__version__),", CUDA version:", torch.version.cuda)

1.5.1+cu101
PyTorch version: None , CUDA version: 10.1


In [3]:
!pip install torch-scatter==latest+cu101 torch-sparse==latest+cu101 -f https://s3.eu-central-1.amazonaws.com/pytorch-geometric.com/whl/torch-1.5.0.html
!pip install torch-geometric

Looking in links: https://s3.eu-central-1.amazonaws.com/pytorch-geometric.com/whl/torch-1.5.0.html
     |████████████████████████████████| 12.3MB 251kB/s 
     |████████████████████████████████| 21.6MB 1.3MB/s 
     |████████████████████████████████| 174kB 4.5MB/s 
     |████████████████████████████████| 235kB 13.8MB/s 
     |████████████████████████████████| 2.1MB 12.8MB/s 
     |████████████████████████████████| 51kB 8.2MB/s 
  Created wheel for torch-geometric: filename=torch_geometric-1.6.0-cp36-none-any.whl size=296339 sha256=7deacd249da5a938b6710db067d4c986f81ec61ffbaedfac4bb9ae76e0d924f9
  Stored in directory: /root/.cache/pip/wheels/5f/7f/33/acea5809d8580a7adf60dcd6d04f5fc50a7f983040f68be1ff
Successfully built torch-geometric


In [4]:
from torch_geometric.data import Data
from torch_geometric.data import DataLoader
import torch.nn as nn
from torch_scatter import scatter_add
import torch.nn.functional as F

import ipywidgets as widgets
from ipywidgets import interact, interact_manual
from IPython.display import clear_output
from IPython.display import HTML, display

%matplotlib inline

# Get rid of RuntimeWarnings, gross
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)

## The Problem

The complexity of the graph depends on the angle cut we put on it. Try increasing max_angle to, say, `(5/6)*np.pi` and the graph should be more busy. The limit is of course `(6/6)*np.pi = pi` where each node will look at the full angle of available possible nodes to form an edge with. While playing with this number, run the next cell to see the proportion of fake edges to true edges (fake/true) on the above graph. This value will be extremely useful later.

## A Simple GNN

### Message Passing GNN

We can write out the full GNN as a class `MPNN_Network`. One can see its behaviour as:
1. Encode (x,y) features as hidden features with an N-layer MLP called `node_encoder`
2. Concatenate these along each edge, and feed the concatenated features into another MLP called `edge_network`
3. Sum the output of `edge_classifier` at each node (that is, each node receives the sum of the "messages" of all connecting edges). This sum is fed into `node_network`
4. Add the hidden features to the previous iteration (this helps to preserve information between messages)
5. Repeat (2) --> (4) n_graph_iters times
6. After the message passing loop, pass the features of each edge through an output classifier network called `edge_classifier`

In [5]:
def make_mlp(input_size, sizes,
             hidden_activation='ReLU',
             output_activation='ReLU',
             layer_norm=False):
    """Construct an MLP with specified fully-connected layers."""

    hidden_activation = getattr(nn, hidden_activation)
    if output_activation is not None:
        output_activation = getattr(nn, output_activation)
    layers = []
    n_layers = len(sizes)
    sizes = [input_size] + sizes
    # Hidden layers

    for i in range(n_layers-1):
        layers.append(nn.Linear(sizes[i], sizes[i+1]))
        if layer_norm:
            layers.append(nn.LayerNorm(sizes[i+1]))
        layers.append(hidden_activation())

    # Final layer
    layers.append(nn.Linear(sizes[-2], sizes[-1]))
    if output_activation is not None:
        if layer_norm:
            layers.append(nn.LayerNorm(sizes[-1]))
        layers.append(output_activation())
    return nn.Sequential(*layers)

class MPNN_Network(nn.Module):
    """
    A message-passing graph network which takes a graph with:
    - bi-directional edges
    - node features, no edge features

    and applies the following modules:
    - a graph encoder (no message passing)
    - recurrent edge and node networks
    - an edge classifier
    """

    def __init__(self, input_dim, hidden_node_dim, hidden_edge_dim, in_layers, node_layers, edge_layers,
                 n_graph_iters=1, layer_norm=True,normalize_factor= 1):
        super(MPNN_Network, self).__init__()
        self.n_graph_iters = n_graph_iters

        # The node encoder transforms input node features to the hidden space
        self.node_encoder = make_mlp(input_dim, [hidden_node_dim]*in_layers)

        # The edge network computes new edge features from connected nodes
        self.edge_network = make_mlp(2*hidden_node_dim,
                                     [hidden_edge_dim]*edge_layers,
                                     layer_norm=layer_norm)

        # The node network computes new node features
        self.node_network = make_mlp(hidden_node_dim + hidden_edge_dim,
                                     [hidden_node_dim]*node_layers,
                                     layer_norm=layer_norm)

        # The edge classifier computes final edge scores
        self.edge_classifier = make_mlp(2*hidden_node_dim,
                                        [hidden_edge_dim, 1],
                                        output_activation=None)
        self.normalize_factor=normalize_factor
    def forward(self, data):
        # Make every edge bi-directional
        send_idx = torch.cat([data.edge_index[0], data.edge_index[1]], dim=0)
        recv_idx = torch.cat([data.edge_index[1], data.edge_index[0]], dim=0)
        
        # Encode the graph features into the hidden space
        #print( torch.mean (data.x[:,0]) )
        #print( torch.mean (data.x[:,1]) )
        if self.normalize_factor!=1:
          temp = data.x.clone().detach()
          temp[:,1] *= self.normalize_factor
          x = self.node_encoder(temp)
        else:
          x = self.node_encoder(data.x.clone().detach())
        # Loop over graph iterations
        for i in range(self.n_graph_iters):

            # Previous hidden state
            x0 = x

            # Compute new edge features
            edge_inputs = torch.cat([x[send_idx], x[recv_idx]], dim=1)
            e = self.edge_network(edge_inputs)

            # Sum edge features coming into each node
            aggr_messages = scatter_add(e, recv_idx, dim=0, dim_size=x.shape[0])

            # Compute new node features
            node_inputs = torch.cat([x, aggr_messages], dim=1)
            x = self.node_network(node_inputs)

            # Residual connection
            x = x + x0

        # Compute final edge scores; use original edge directions only
        start_idx, end_idx = data.edge_index
        clf_inputs = torch.cat([x[start_idx], x[end_idx]], dim=1)
        return self.edge_classifier(clf_inputs).squeeze(-1)

Build a version of the model and print it out.

In [6]:
#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#m_configs = {"input_dim": 2, "hidden_node_dim": 64, "hidden_edge_dim": 64, "in_layers": 2, "node_layers": 4, "edge_layers": 4, "n_graph_iters": 1, "layer_norm": True}
#model = MPNN_Network(**m_configs).to(device)

### Training and Validation Functions


In [7]:
def train(model, train_loader, optimizer):
    correct = 0
    total = 0
    total_loss = 0
    for batch in train_loader:
        optimizer.zero_grad()
        data = batch.to(device)
        pred = model(data)
        loss = F.binary_cross_entropy_with_logits(pred.float(), data.y.float(), pos_weight=torch.tensor(weight))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        correct += ((pred > 0.5) == (data.y > 0.5)).sum().item()
        total += len(pred)
    acc = correct/total
    return acc, total_loss

def evaluate(model, test_loader):
    correct = 0
    total = 0
    total_loss = 0
    for batch in test_loader:
        data = batch.to(device)
        pred = model(data)
        loss = F.binary_cross_entropy_with_logits(pred.float(), data.y.float(), pos_weight=torch.tensor(weight))
        total_loss += loss.item()
        correct += ((pred > 0.5) == (data.y > 0.5)).sum().item()
        total += len(pred)
    acc = correct/total
    return acc, total_loss

We set a weight value that is more or less the (fake / true) ratio found above. This forces the loss function to punish incorrectly classified true edges more severely. It rebalances the distribution as if there was a 1:1 true:fake ratio.

### Did it work?

Running the above with 1 graph iteration gives me about 90% accuracy in 200 epochs.

The best performance that I can get with some simple manual tuning is around 95% accuracy. We can improve the efficiency at the cost of purity by raising the weight on true edges, but the accuracy won't significantly improve. In general, the biggest changes were from increasing the width (i.e. the number of dimensions) of the hidden layers. We can visualise the performance on a particular graph, colouring true positives black, false positives red, true negatives a transparent black, and false negatives in blue.

In [8]:
def plot_toy_classified(event, preds, cut=0.5):
    
    #print("New event")
        
    #print(event)
    #print(event.x)
    #print(event.pid)
    #print(event.y)
    #gprint(preds)

    plt.figure(figsize=(10,5))
    x, y = event.x[:,0].numpy(), event.x[:,1].numpy()
    edges = event.edge_index.numpy()
    labels = event.y
    plt.scatter(x, y, c='k')
    
    preds = preds.detach().numpy()
    

    for j in range(len(labels)):
        
        #print(str('%01.2f' % preds[j])+" "+str(labels[j].item())+" "+str(x[edges[0,j]]*feature_scale)+","+str(y[edges[0,j]]*feature_scale)+" "+str(x[edges[1,j]]*feature_scale)+","+str(y[edges[1,j]]*feature_scale))
        s = str('%01.2f' % preds[j])+" "+str(labels[j].item())
        plt.text((x[edges[0,j]]+x[edges[1,j]])/2., (y[edges[0,j]]+y[edges[1,j]])/2.+np.random.random(1)*0.00, s, fontsize=12)

        # False negatives
        if preds[j] < cut and labels[j].item() > cut:
            plt.plot([x[edges[0,j]], x[edges[1,j]]],
                     [y[edges[0,j]], y[edges[1,j]]],
                     '--', c='b')

        # False positives
        if preds[j] > cut and labels[j].item() < cut:
            plt.plot([x[edges[0,j]], x[edges[1,j]]],
                     [y[edges[0,j]], y[edges[1,j]]],
                     '-', c='r', alpha=preds[j])

        # True positives
        if preds[j] > cut and labels[j].item() > cut:
            plt.plot([x[edges[0,j]], x[edges[1,j]]],
                     [y[edges[0,j]], y[edges[1,j]]],
                     '-', c='k', alpha=preds[j])
                
        # True negatives
        if preds[j] < cut and labels[j].item() < cut:
            plt.plot([x[edges[0,j]], x[edges[1,j]]],
                     [y[edges[0,j]], y[edges[1,j]]],
                     '-', c='k', alpha=preds[j])

So we can see that it's working quite well. Few missed true edges, and few misclassified fake edges. The ratio of false positives to false negatives (which can be defined with efficiency and purity) is controlled by the cut we put on the prediction score. 

### The effect of Message Passing

In this simple example, the message passing does not do a huge amount. Going from 1 iteration to 6 iterations improves the accuracy to around 93% (from 90%). This improvement is washed out with more hidden dimensions, as one can see from the below set of tests. Try playing with more hidden node and edge dimensions (e.g. 64) and see if the message passing iterations can improve the performance.



# **Load MuonE data and reconstruct tracks**

In [9]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [10]:
# Circle parameters
num_layers = 14
height, width = 10, 10
#min_curve, max_curve =  1000, 1001 # 15, 50

max_angle = (0.05)*np.pi
feature_scale = 200.

In [14]:
import numpy as np
import pandas as pd

  
def readMuonE(event_file,is_x=True):


   data = pd.read_csv(event_file) 
   # Preview the first 5 lines of the loaded data 
   data.head()

  #if axis_oi == 0 we are getting the data from the X detector layers
  #if axis oi == 1, it's about the data from Y detector layers
   if is_x:
    axis_oi=0

   else:
    axis_oi=1

   
   
   
   # Temporary nextZ  (next layer Z)
   layerZ0 =  [-999, -16, -21, 131, 151, 477, 497, 832, 843.7, 1146, 1141, 1252.7, 1266, 1973, 1960 ]



   #X = np.array([[0,0,0]])
   #print(X)
   
   
   iEventLast=data.iat[0,0]
   k=0
   Xcreated=False
   #dataOUTcreated=False
   dataOUT = []

   #iterate over input data
   for k, row in data.iterrows():
     ll=data.iat[k,0]
     #print("counter ",k,ll,iEventLast)
     if ll != iEventLast:

       '''
       for layer in np.arange(num_layers-1):
          print("XX=",layer,X)
          for i in np.argwhere(X[:,2] == layerZ0[layer+1]): 
            for j in np.argwhere(X[:,2] == layerZ0[layer+3]):
               if (X[i, 0] - 20. < X[j, 0] < X[i, 0] + 20.):
                   print("layer, i, j=",layer, i, j)
       '''
#       e = np.array([[i,j] for layer in np.arange(num_layers-1) for i in np.argwhere(X[:,2] == layer+1) for j in np.argwhere(X[:,2] == (layer+2))  if (X[i, 0] - np.tan(max_angle/2) < X[j, 0] < X[i, 0] + np.tan(max_angle/2))]).T.squeeze()

       e = np.array([[i,j] for layer in np.arange(num_layers-1) for i in np.argwhere(X[:,2] == 
              layerZ0[layer+1]) for j in np.argwhere(X[:,2] == layerZ0[layer+3])  if (X[i, 0] - np.tan(max_angle/2)*(layerZ0[layer+3]-layerZ0[layer+1]) < X[j, 0] < X[i, 0] + np.tan(max_angle/2)*(layerZ0[layer+3]-layerZ0[layer+1]))]).T.squeeze()

       #print(X)
       #print("e ",e) 
        # This handles when no edges were constructed. In that case, the randomisation is a do-over
       #try:
       y = np.array([int(i[1] == j[1]) for i,j in zip(X[e[0]], X[e[1]])])
       #     print("y = ",y)     
       #     break
       #except:
       #     pass

       #print("y = ",y) 
       #if iter is not None and num_samples is not None:
       #     out.update(progress(iter, num_samples))    


       X = np.array([X[:,2], X[:,0]]).T / feature_scale

       dataEvent = Data(x = torch.from_numpy(X).float(), edge_index = torch.from_numpy(e), y = torch.from_numpy(y), pid = torch.from_numpy(X[:,1])  )

       #plot_toy_graph(dataEvent, 0.2)

       dataOUT.append(dataEvent)
#       if dataOUTcreated:
#         dataOUT = dataOUT.append(dataEvent)
#       else:
#         dataOUT = dataEvent 
#         dataOUTcreated = True

       Xcreated = False  
       iEventLast = ll

     #print(len(X)) 
     #print(k,ll,data.iat[k,2],Xcreated) 
     if data.iat[k,2]==axis_oi and data.iat[k,5]<3 and data.iat[k,5]>=0: #remove equal to part if you don't want to  # select X-hit and trackID=0,1,2
        if Xcreated:
          X = np.append(X,np.array([[data.iat[k,1], data.iat[k,5], data.iat[k,3]  ]]),axis=0)
        else:
          X = np.array([[data.iat[k,1], data.iat[k,5], data.iat[k,3]  ]]) 
          Xcreated=True
          #print("X created ",X)
     
   return dataOUT       





#Reads original monte carlo File

def readMCtrack(event_file, is_x= True ):

  data = pd.read_csv(event_file) 
  # Preview the first 5 lines of the loaded data 
  data.head()

  if is_x:
    axis_oi=[4,1]
  else  :
    axis_oi=[5,2]

  temp=data.iat[-1,0]
  track_slopes=torch.zeros(temp,6)

  for k, row in data.iterrows():

    if (data.iat[k,7]==np.array([0,1,2])).any():

      

      if track_slopes.shape[0]< data.iat[k,0]-1 :
        track_slopes=torch.cat([track_slopes,torch.zeros(1,6)],dim=0)
      
      
      track_slopes[data.iat[k,0]-1,data.iat[k,7]*2 ] = data.iat[k,axis_oi[0]]/data.iat[k,6]

      track_slopes[data.iat[k,0]-1,data.iat[k,7]*2+1] =  data.iat[k,axis_oi[1]] - data.iat[k,axis_oi[0]]/data.iat[k,6]  * data.iat[k,3]
  
  return track_slopes










# Reads From Reconstruction File

def ReadRecon(event_file,is_x=True):
  data = pd.read_csv(event_file)  
  data.head()

  if is_x:
    axis_oi=[3,1,7,5,9]
  else:
    axis_oi=[4,2,8,6,9]

  track_=torch.zeros(data.iat[-1,0],10)

  for k,row in enumerate(data.iterrows()):

    if k % 2 ==0:
      #track_[data.iat[k,0]-1,0],track_[data.iat[k,0]-1,1]=data.iat[k,axis_oi[0]],data.iat[k,axis_oi[1]]
      for i in range( len(axis_oi) ):
        track_[data.iat[k,0]-1,i]=data.iat[k,axis_oi[i]]
    else:
      #track_[data.iat[k,0]-1 ,2],track_[data.iat[k,0]-1,3]=data.iat[k,axis_oi[0]],data.iat[k,axis_oi[1]]
      for i in range( len(axis_oi) ):
        track_[data.iat[k,0]-1,i+5]=data.iat[k,axis_oi[i]]
  print(k)
  

  temp=track_.clone()
  for i in range(data.iat[-1,0]):
    if torch.abs(temp[i,0])>torch.abs(temp[i,5]):

      track_[i,0:4]=temp[i,5:9]
      track_[i,5:9]=temp[i,0:4]



  return track_





In [13]:
# Read data and prepare for training

is_x = True
is_y = False
event_file_MC= "/content/drive/My Drive/MuonE_tracking/MCtrackFile_medium.csv"
event_file= "/content/drive/My Drive/MuonE_tracking/hitFile_medium.csv"
event_file_recon= "/content/drive/My Drive/MuonE_tracking/trackFile_medium.csv"

Tracks=readMCtrack(event_file_MC,is_x)
data = readMuonE(event_file,is_x)
Tracks_recon=ReadRecon(event_file_recon,is_x)

#Tracks_x=readMCtrack(event_file_MC,is_x)
#data_x = readMuonE(event_file,is_x)
#Tracks_recon_x=ReadRecon(event_file_recon,is_x)

# split data into training and test datasets
train_dataset,test_dataset,test_tracks,train_tracks,train_recon,test_recon = [],[],[],[],[],[]

for k in range(len(data)):
  if (k%4 == 0):
    test_dataset.append(data[k])
    test_tracks.append(Tracks[k,:])
    test_recon.append(Tracks_recon[k,:])
  else:
    train_dataset.append(data[k])
    train_tracks.append(Tracks[k,:])
    train_recon.append(Tracks_recon[k,:])
train_loader = DataLoader(train_dataset, batch_size=100, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=100, shuffle=True)    

# Display few events
#for k in range(10):
 #  plot_toy_graph(train_loader.dataset[k], 0.2)


print("Fake / True = ", (len(train_dataset[0].y) - train_dataset[0].y.sum().item()) / train_dataset[0].y.sum().item())
print("Fake / True = ", (len(test_dataset[0].y) - test_dataset[0].y.sum().item()) / test_dataset[0].y.sum().item())

print("Training dataset (events): ",len(train_dataset))
print("Test dataset (events): ",len(test_dataset))

19073
Fake / True =  1.0
Fake / True =  0.875
Training dataset (events):  7499
Test dataset (events):  2500


In [ ]:
def plot_toy_graph(event, ylim=0.):
  
    #print(event)
    #print(event.x)
    #print(event.pid)
    #print(event.y)

    plt.figure(figsize=(10,5))
    x, y = event.x[:,0].numpy(), event.x[:,1].numpy()
    e = event.edge_index.numpy()
#     for i, j in zip(X[e[0]], X[e[1]]):
#         plt.plot([i[0], j[0]], [i[1], j[1]], c='b')
    #     print(i[0], i[2], j[0], j[2])
    plt.plot([x[e[0,:]], x[e[1,:]]], [y[e[0,:]], y[e[1,:]]], c='b')
    plt.scatter(x, y, c='k')
    if (ylim!=0):
       plt.ylim(-ylim,ylim)
    else:
       diff = 0.1*(max(y)-min(y))
       plt.ylim(min(y)-diff, max(y)+diff)   

In [ ]:

# Display few events
for k in range(10):
   plot_toy_graph(train_loader.dataset[k], 0.)

print(train_loader.dataset[k].x)

print("Fake / True = ", (len(train_dataset[0].y) - train_dataset[0].y.sum().item()) / train_dataset[0].y.sum().item())

print("Training dataset (events): ",len(train_dataset))
print("Test dataset (events): ",len(test_dataset))

# **Simple GNN**

We set a weight value that is more or less the (fake / true) ratio found above. This forces the loss function to punish incorrectly classified true edges more severely. It rebalances the distribution as if there was a 1:1 true:fake ratio.

In [ ]:
t_loss_v = []
t_acc_v = []
v_loss_v = []
v_acc_v = []
ep = 0

weight = 1
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#device = torch.device('cpu')
m_configs = {'normalize_factor': 2000 ,"input_dim": 2, "hidden_node_dim": 64, "hidden_edge_dim": 64, "in_layers": 1, "node_layers": 2, "edge_layers": 2, "n_graph_iters": 6, "layer_norm": True}
model = MPNN_Network(**m_configs).to(device)

#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#m_configs = {"input_dim": 2, "hidden_node_dim": 64, "hidden_edge_dim": 64, "in_layers": 2, "node_layers": 4, "edge_layers": 4, "n_graph_iters": 1, "layer_norm": True}
#model = MPNN_Network(**m_configs).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, weight_decay=1e-3, amsgrad=True)

for epoch in range(52):
    ep += 1  
    model.train()
    acc_t, total_loss = train(model, train_loader, optimizer)
    t_loss_v.append(total_loss)
    t_acc_v.append(acc_t)

    model.eval()
    acc, total_loss = evaluate(model, test_loader)
    v_loss_v.append(total_loss)
    v_acc_v.append(acc)
    if ep % 5 ==1 :
      print('Epoch: {}, Accuracy: {:.4f}'.format(ep, acc,acc_t))



In [ ]:
fig, axs = plt.subplots(1, 2, constrained_layout=True, figsize=(10, 5))

axs[0].plot(np.arange(len(t_loss_v)), t_loss_v)
axs[0].twinx().plot(np.arange(len(t_acc_v)), t_acc_v,"r")
axs[0].set_title("Training loss and accuracy")
axs[0].set_yscale("linear")
axs[1].plot(np.arange(len(v_loss_v)), v_loss_v)
axs[1].twinx().plot(np.arange(len(v_acc_v)), v_acc_v,"r")
axs[1].set_title("Validation loss and accuracy")
axs[1].set_yscale("linear")

### Did it work?

In [ ]:


for i in range(0,1500):
  data = test_loader.dataset[i].to(device)
  preds = torch.sigmoid(model(data)).to('cpu')  
  
  a=0
  for j in range(preds.shape[0]):
    if preds[j] > 0.5 and preds[j] < 0.8 and 0 :   
      a=1
  if a==1:
    plot_toy_classified(data.to('cpu'), preds, cut = 0.6)
  

  #plot_toy_graph(train_loader.dataset[2],0.2)


In [ ]:
from sklearn.linear_model import (
    LinearRegression, TheilSenRegressor, RANSACRegressor, HuberRegressor)
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn import linear_model

from scipy.optimize import curve_fit, least_squares
from scipy import stats


def finder(data,y,cut=0.5):
  x = data.x.clone().numpy()
  edge_index=data.edge_index.detach().numpy()
  node_identifier=np.zeros( list(x.shape)[0] )
  k=edge_index.shape[1]
  y=y.detach().tolist()
  is_connected=[]
  for i in range(k):
    if y[i]-cut>=0:
      is_connected.append( True )
    else:
      is_connected.append( False )

  for j in range(1,4):

    a=np.where( node_identifier==0 )
    if j==1:
      try:
        min=np.min(np.abs(x[a,0]))
      except:
        print('oopsie')
        break
      idx=np.where(np.abs(x[:,0])==min)
    

    else:
      try:
        min=np.min(np.abs(x[a,1]))
      except:
        print('oopsie')
        break
      idx=np.where(np.abs(x[:,1])==min)
    
    try:
      idx=idx[0]
    except:
      pass
    
    neighbor = 1
    node_identifier[idx] = j
    
    
    while neighbor :

      neighbor = []
      try:
        idx=idx[-1]
      except:
        pass
      in_neighbor = [ int(edge_index[0,i]) for i in range(k) if int(edge_index[1,i])==idx if is_connected[i] 
                     if node_identifier[ edge_index[0,i] ] ==0  ]

      ot_neighbor = [ int(edge_index[1,i]) for i in range(k) if int(edge_index[0,i])==idx if is_connected[i] 
                     if node_identifier[ edge_index[1,i] ] ==0  ]
      neighbor = in_neighbor + ot_neighbor
      
      if len(neighbor)>0:
        for i in reversed( range( len(neighbor) ) ):
          if node_identifier[neighbor[i]] != 0 :
            del neighbor[i]
      

      if neighbor:
        idx=neighbor[0]

      node_identifier[neighbor] = j



  return node_identifier





def fit(x,a,b): 
  return a*x+b
def chi2(x,y,yerr,fit,popt):
  # Now calculate chi square 
  yexp = fit(x, *popt)
  r_ = (y - yexp)/yerr
  chisq_ = np.sum(r_**2)
  df_ = len(x) - 2
  return chisq_, df_, r_



def robust_fit(data,preds,toleration=0.5):
  node_lines = finder(data,preds)
  x,y=data.x[:,0].clone().numpy()*feature_scale,data.x[:,1].clone().numpy()*feature_scale
  
  parameters=[]
  errors=[]
  popt=[]
  pcov=[]
  for i in range(2,4):
    
    nodes=np.where(node_lines==i)
    
    if np.size(nodes)>1 :

      yerr = np.ones( np.size(nodes) )*0.03
      popt,pcov=curve_fit(fit,x[nodes],y[nodes],p0=(0.0,0.0),sigma=yerr, absolute_sigma=True) 

      a=popt[0]
      err_a=np.sqrt(pcov[0,0])
      b=popt[1]
      err_b=np.sqrt(pcov[1,1])
      chisq_limit = toleration 

      chisq, df, r = chi2(x[nodes],y[nodes],yerr,fit,popt)
      chisq_=[]
      if int(chisq/df > chisq_limit) and np.size(nodes)>3  :
        for ii in range(np.size(nodes)):

          x_robust = np.delete(x[nodes],ii)
          y_robust = np.delete(y[nodes],ii)
          yerrii = np.delete(yerr,ii)
          popt,pcov=curve_fit(fit , x_robust , y_robust , p0=(0.0,0.0),sigma=yerrii, absolute_sigma=True)
          chisq, df, r = chi2(x_robust,y_robust,yerrii,fit,popt)      
          chisq_.append(chisq)

        ii=np.where(np.array(chisq_)==np.amin(np.array(chisq_)))[0] 
        
        x_robust = np.delete(x[nodes],ii)
        y_robust = np.delete(y[nodes],ii)
        yerr = np.delete(yerr,ii)
        popt,pcov=curve_fit(fit , x_robust , y_robust , p0=(0.0,0.0),sigma=yerr, absolute_sigma=True)
        chisq, df, r = chi2(x_robust,y_robust,yerr,fit,popt)      
          



      #if int(chisq/df > chisq_limit) and np.size(nodes)>3  :
    # Find outlier and remove it

    # find point with the highest abs(r)
        #ii = np.where( np.abs(r) == np.amax(abs(r)))[0]

        #x_robust = np.delete(x[nodes],ii)
        #y_robust = np.delete(y[nodes],ii)
        #yerr = np.delete(yerr,ii)
    # repeat fit without outlier
        #popt,pcov=curve_fit(fit , x_robust , y_robust , p0=(0.0,0.0),sigma=yerr, absolute_sigma=True)
        #chisq, df, r = chi2(x_robust,y_robust,yerr,fit,popt)      


      a=popt[0]
      err_a=np.sqrt(pcov[0,0])
      b=popt[1]
      err_b=np.sqrt(pcov[1,1])    
      parameters.append([a,b])
      errors.append([err_a,err_b,yerr,chisq])

    
    else :
      errors.append('N/A')
      parameters.append('N/A')

  return parameters,errors,node_lines



def plot_classified_line(data,preds,Track,trackre ):

  parameters_,errors_,node_lines=robust_fit(data,preds)
  x,y=data.x[:,0]*feature_scale,data.x[:,1]*feature_scale
  colors={'1':'r','2':'g','3':'b' }
  fig=plt.figure(figsize=(10,10))
  ax=fig.add_subplot(111)

  if (node_lines==0).any()== True:
    nodes=np.where(node_lines==0)
    plt.plot(x[nodes],y[nodes], 'k'+'o' , label = 'Not Track')
    
  for i in range(1,4):
    
    nodes=np.where(node_lines==i)
    plt.plot(x[nodes],y[nodes],colors[str(i)] +'o' )
    nodes=np.where(node_lines==i)


  for i in [2,3]:
    
    nodes=np.where(node_lines==i)
    


    if np.size(nodes)>1 :
      
      error=errors_[i-2]  
      

      x_guess=np.linspace(0,torch.max(x[nodes]),100)
      x_line=np.linspace(torch.min(x[nodes]),torch.max(x[nodes]),100)

      
      
      
      slope_gnn=parameters_[i-2][0]
      c_gnn=parameters_[i-2][1] 





      slope_recon_ = trackre[5*i-10]
      c_recon = trackre[5*i-9]  
      error_recon_ = trackre[5*i-8]
      


      slope_mc = Track[2*i-2]
      #c_mc = Track[2*i-1]
      c_mc=y[nodes][1]-slope_mc*x[nodes][1]

      # Plotting of the GNN
      

      
      plt.plot(x_guess,c_gnn + torch.tensor(x_guess)*slope_gnn,'--'+colors[str(i)],label="GNN{} Error: {:e}, ".format(i-1, 
          torch.abs( np.round( ( slope_mc - slope_gnn ) /error[0] ,2) ).tolist() ))
      


      # Plotting of original MC tracks


      
      
      plt.plot(x_guess,c_mc + torch.tensor(x_guess)*slope_mc,'k',label='MC Track', alpha=0.4) 

      # Plotting of the regular reconstruction

      
      plt.plot( torch.tensor(x_line), torch.tensor(x_line) * slope_recon_ + c_recon ,colors[str(i)], label=r"recon{} Error: {:e}, ".format(i-1, 
          torch.abs( np.round( (slope_mc-slope_recon_)/error_recon_ ,2) ).tolist() )  )   
      

            
      ax.errorbar(x[nodes],y[nodes],yerr= np.ones(np.size(nodes,1) )*0.03,fmt='none', ecolor=colors[str(i)] )
  plt.legend()


In [ ]:
#for i in range(len(train_loader.dataset)):
for i in range(2,10):

  data = train_loader.dataset[i].to(device) 
  preds=torch.sigmoid(model(data))
  errors=robust_fit(data.to('cpu'),preds)[0]
  a = errors[1][0]
  b = errors[0][0]
  if not (isinstance (a,str) or isinstance (b,str) ):
    if True or b > 0.04 or b < -0.03 or a > 0.04 or a < -0.03  :
      plot_classified_line(data.to('cpu'),preds,train_tracks[i], train_recon[i] )
      plt.title(str(i))

In [ ]:
from scipy.stats import norm

error_recon,error_gnn,error_recon_nopull,error_gnn_nopull=[],[],[],[]

for iter in range (0,len(test_tracks)):

  data = test_loader.dataset[iter].to(device)
  preds = torch.sigmoid(model(data))
  temp = robust_fit(data.to('cpu'),preds)
  slope = temp[0]

  if not (slope[0] == 'N/A' or slope[1] == 'N/A' ) :

    error_gnn.append( ((test_tracks[iter][2]-slope[0][0])/temp[1][0][0] )  )
    error_gnn.append( ((test_tracks[iter][4]-slope[1][0])/temp[1][1][0] ))
    
    error_gnn_nopull.append( ((test_tracks[iter][2]-slope[0][0]) ).numpy() )
    error_gnn_nopull.append( ((test_tracks[iter][4]-slope[1][0]) ).numpy() )
  
  error_recon.append( ((test_tracks[iter][2]-test_recon[iter][0]))/test_recon[iter][2]  )
  error_recon.append( ((test_tracks[iter][4]-test_recon[iter][5]))/test_recon[iter][7] )

  error_recon_nopull.append( ((test_tracks[iter][2]-test_recon[iter][0])) )
  error_recon_nopull.append( ((test_tracks[iter][4]-test_recon[iter][5])))


fig=plt.figure(figsize=(15,15))

limit_1=0.001
limit_2=4
error_list = { 'Reconstructed,  True-Predicted ' : error_recon_nopull,'GNN, True-Predicted ' : error_gnn_nopull,
              'Reconstructed, (True-Predicted)/error' : error_recon,'GNN, (True-Predicted)/error' : error_gnn}            



for i,[title,fitdata] in enumerate(error_list.items()):

  ax_list=fig.add_subplot(2,2,i+1)
  if i<2:
    limit=limit_1
  else:
    limit=limit_2
  supress_=ax_list.hist(np.array(fitdata),bins=np.linspace(-limit,limit,100).tolist(), density=True)

  if i>1:
    #fit gaussian
    #remove infinites from fitdata 
    fitdata2 = []
    for k in range(len(fitdata)):
      if fitdata[k].item()<2 and fitdata[k].item()>-2:
        fitdata2.append(fitdata[k].item())

    mean,std=norm.fit(fitdata2)
    print(mean,std,fitdata2)
    xmin, xmax = plt.xlim()
    x = np.linspace(xmin, xmax, 100)
    y = norm.pdf(x, mean, std) 
    plt.plot(x, y)
    title = title+" mean = %.2f,  std = %.2f" % (mean, std)
    plt.title(title)
  else:
    plt.title(title)


 

fig.show()



Don't Run! Old Robust fit algorithm


In [ ]:
#@title Default title text
print(fdgldfc)
def robust_fit(data,preds,estimator=linear_model.RANSACRegressor() ):
  node_lines = finder(data,preds)
  x,y=data.x[:,0],data.x[:,1]
  errors,models=[],[]
  parameters=[]
  for i in range(1,4):
    
    nodes=np.where(node_lines==i)
    
    if np.size(nodes)>1 :
      
      model = linear_model.RANSACRegressor()
      model.fit( x[nodes].reshape(-1, 1)  , y[nodes] )
      
      parameters.append(  [ model.estimator_.coef_, model.predict(  np.zeros(1).reshape(-1,1) ) ]) 
      error = mean_squared_error( model.predict(x[nodes].reshape(-1, 1)), y[nodes] )
      error= np.round(error,2)
      errors.append( error )
      models.append( model )
    
    else  :

      models.append('N/A')
      errors.append('N/A')
      parameters.append('N/A')
  
  return [models,errors,parameters,node_lines]

def plot_classified_line(data,preds,Track,trackre ):

  models,errors,parameters,node_lines=robust_fit(data,preds,linear_model.RANSACRegressor())
  x,y=data.x[:,0],data.x[:,1]
  colors={'1':'r','2':'g','3':'b' }
  fig=plt.figure(figsize=(10,10))
  ax=fig.add_subplot(111)

  if (node_lines==0).any()== True:
    nodes=np.where(node_lines==0)
    plt.plot(x[nodes],y[nodes], 'k'+'o' , label = 'Not Track')
  for i in range(1,4):
    
    nodes=np.where(node_lines==i)
    plt.plot(x[nodes],y[nodes],colors[str(i)] +'o' )

  for i in [2,3]:
    
    nodes=np.where(node_lines==i)
    


    if np.size(nodes)>1 :
      
      model=models[i-1]
      error=errors[i-1]
      
      x_guess=np.linspace(0,torch.max(x[nodes]),100)
      y_guess = model.predict( x_guess[:, np.newaxis] )
      x_line=np.linspace(torch.min(x[nodes]),torch.max(x[nodes]),100)

      slope_mc=Track[i-1]
      c_mc = y[nodes][0] - x[nodes][0] * slope_mc
      slope_recon_ = trackre[2*i-4]
      c_recon= y[nodes][0] - x[nodes][0] * slope_recon_ 

      plt.plot(x_guess,y_guess,'--'+colors[str(i)],label="GNN{} Error: {:e}, ".format(i-1, 
          torch.abs( np.round( (slope_mc-parameters[i-1][0])/slope_mc ,2) ).tolist()[0] ))
      

      plt.plot( torch.tensor(x_line), torch.tensor(x_line) * slope_recon_ + c_recon ,colors[str(i)], label="recon{} Error: {:e}, ".format(i-1, 
          torch.abs( np.round( (slope_mc-slope_recon_)/slope_mc ,2) ).tolist() )  )   

#. format( error , np.round(parameters[i-1][0],2),np.round(parameters[i-1][1],2)   )   )
      
      
      plt.plot(x_line,c_mc + torch.tensor(x_line)*slope_mc,'k',label='MC Track', alpha=0.4) 
      
            
      #ax.errorbar(x[nodes],y[nodes],yerr= np.ones(np.size(nodes,1) )*0.03,fmt='none', ecolor=colors[str(i)] )
  plt.legend()


In [ ]:
train_tracks[1008]